<a href="https://colab.research.google.com/github/MoonlightO2/UK-Weather-Engineering-Analysis-/blob/main/UK_Weather_(Web_Scrape).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **UK Weather (Web Scrape)**

### *Datasets*:
**weather_csv** = '/content/drive/MyDrive/2. MSc/Colab Notebooks/Portfolio/Daily task - Web Scraping News - Data Analysis/World Weather Data Collection/World Weather.csv'

**locations_csv** = '/content/drive/MyDrive/2. Projects/Colab Notebooks/Portfolio/Daily task - Web Scraping News - Data Analysis/World Weather Data Collection/UK Locations Data.csv'

In [1]:
import requests
import csv
from datetime import datetime
import pandas as pd

In [2]:
weather_csv = '/content/drive/MyDrive/2. Projects/Colab Notebooks/Portfolio/Daily task - Web Scraping News - Data Analysis/World Weather Data Collection/UK Weather.csv'
locations_csv = '/content/drive/MyDrive/2. Projects/Colab Notebooks/Portfolio/Daily task - Web Scraping News - Data Analysis/World Weather Data Collection/UK Locations Data.csv'

In [3]:
df = pd.read_csv(locations_csv)

In [4]:
print('Shape:', df.shape, '\n\n')
df.head()

Shape: (647, 3) 




,City,Latitude,Longitude
0,Abbots Langley,51.7010,-0.4160
1,Aberaman,51.7000,-3.4333
2,Aberbargoed,51.6968,-3.2240
3,Aberdeen,57.1500,-2.1100
4,Abingdon,51.6670,-1.2830


In [5]:
# Rename columns
df.rename(columns={
    'city': 'City',
    'lat': 'Latitude',
    'lng': 'Longitude',
    'country': 'Country',
    'iso2': 'Country Code',
    'admin_name': 'City Name',
    'capital': 'Primary',
    'population': 'Population',
    'population_proper': 'Real Population'
}, inplace=True)

In [6]:
print('Shape:', df.shape, '\n\n')
df.head()

Shape: (647, 3) 




,City,Latitude,Longitude
0,Abbots Langley,51.7010,-0.4160
1,Aberaman,51.7000,-3.4333
2,Aberbargoed,51.6968,-3.2240
3,Aberdeen,57.1500,-2.1100
4,Abingdon,51.6670,-1.2830


In [7]:
def get_weather_data(api_key, city):
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        'q': f'{city}',
        'appid': api_key,
        'units': 'metric'  # Use 'imperial' for Fahrenheit
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    if response.status_code == 200:
        date_time_utc = datetime.utcfromtimestamp(data['dt'])
        date = date_time_utc.strftime('%Y-%m-%d')
        time = date_time_utc.strftime('%H:%M:%S')

        temperature = data['main']['temp']
        humidity = data['main']['humidity']
        precipitation = data.get('rain', {}).get('1h', 0)  # 1-hour precipitation in mm
        wind_speed = data['wind']['speed']
        latitude = data['coord']['lat']
        longitude = data['coord']['lon']

        return date, time, city, latitude, longitude, temperature, humidity, precipitation, wind_speed
    else:
        print(f"Error getting data for {city}: {data.get('message', 'Unknown error')}")
        return None

def write_to_csv(file_path, data, fieldnames):
    with open(file_path, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Check if the file is empty and write the header
        if csvfile.tell() == 0:
            writer.writeheader()

        writer.writerow(dict(zip(fieldnames, data)))

def main():
    api_key = 'ed1a7ece48676f3646181376432fa2ba'

    with open(locations_csv, 'r', encoding='utf-8', errors='ignore') as locations_file:
        locations_reader = csv.DictReader(locations_file)

        # Print headers to check the column names
        print(locations_reader.fieldnames)

        for location in locations_reader:
            city = location.get('City')  # Use get() to handle missing keys gracefully

            if city:
                weather_data = get_weather_data(api_key, city)

                if weather_data:
                    date, time, city, latitude, longitude, temperature, humidity, precipitation, wind_speed = weather_data
                    row_data = [date, time, city, latitude, longitude, temperature, humidity, precipitation, wind_speed]


                    # Define the column names in the same order as row_data
                    fieldnames = ['Date', 'Time', 'City', 'Latitude', 'Longitude', 'Temperature', 'Humidity', 'Precipitation', 'Wind Speed']

                    write_to_csv(weather_csv, row_data, fieldnames)
            else:
                print(f"Skipping row due to missing 'City' {city}")

        print(f'File {weather_csv} created using {locations_csv} successfully on {date} at {time}')

In [8]:
if __name__ == "__main__":
  main()

['City', 'Latitude', 'Longitude']
File /content/drive/MyDrive/2. Projects/Colab Notebooks/Portfolio/Daily task - Web Scraping News - Data Analysis/World Weather Data Collection/UK Weather.csv created using /content/drive/MyDrive/2. Projects/Colab Notebooks/Portfolio/Daily task - Web Scraping News - Data Analysis/World Weather Data Collection/UK Locations Data.csv successfully on 2024-01-08 at 22:15:08
